In [1]:
!pip install rfpimp

In [2]:
#!wget https://www.mlgraz.at/course_files/week4/loan.csv

In [3]:
import pandas as pd
from sklearn.impute import SimpleImputer
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from  sklearn.preprocessing import OrdinalEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_validate
from  sklearn.metrics import *
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn import tree
from sklearn import model_selection
import plotly.express as px

In [4]:
import rfpimp

C:\Users\Develop\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:144: FutureWarning:

The sklearn.ensemble.forest module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.



In [5]:
df = pd.read_csv("./data/loan.csv")

# EDA

In [6]:
df.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


**How many rows are there?**

In [ ]:
df.shape

**How many Loans are there in total?**

In [ ]:
df.Loan_ID.nunique()

**How many loans are accepted vs not?**

In [ ]:
px.bar(data_frame=df, x="Loan_Status")

## Relationship with Loan Status

Relationship between Loan and Gender

In [ ]:
px.histogram(df, x="ApplicantIncome", color="Loan_Status")

In [ ]:
df.groupby(["Gender", "Loan_Status"]).count()["Loan_ID"]

# Dealing with Missing Values

**Impute Missing Values (except Column LoanAmount) with the most frequent Values**

In [ ]:
categorical_columns_missing = ["Gender", "Married", "Dependents", "Credit_History", "Self_Employed", "Loan_Amount_Term"]
impute_missing=SimpleImputer(missing_values=np.NaN, strategy='most_frequent')
df[categorical_columns_missing] = impute_missing.fit_transform(df[categorical_columns_missing])

**Impute LoanAmount with the mean**

In [ ]:
impute_missing=SimpleImputer(missing_values=np.NaN, strategy='mean')
df["LoanAmount"] = impute_missing.fit_transform(df[["LoanAmount"]])

# Encoding Categorical Data

In [ ]:
categorical_columns = ["Gender", "Married", "Dependents", "Education", "Self_Employed", "Loan_Status", "Credit_History", "Property_Area"]

**Set Datatype of Ordinal/Nominal Variables to Category**

In [ ]:
for col in categorical_columns:
    df[col] = df[col].astype('category')

**Drop Loan ID**

In [ ]:
df = df.drop("Loan_ID", axis=1)

**Create One Hot Encoding Features for the Categorical Variables**

In [ ]:
df = pd.get_dummies(df, drop_first=True)

In [ ]:
df.head()

**Rename Last Column to Load**

In [ ]:
df.columns = list(df.columns.values[:-1]) + ["Loan"] 

# Split Input and Output

In [ ]:
X = df[df.columns[:-1]]
y = df[df.columns[-1:]]

# Decision Tree

In [ ]:
model=DecisionTreeClassifier()

In [ ]:
model.fit(X,y)

In [ ]:
fig, axes = plt.subplots(nrows = 1,ncols = 1,figsize = (4,4), dpi=300)
tree.plot_tree(model)
fig.savefig('tree.png')

In [ ]:
model.score(X,y)

# Traint Test Split


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [ ]:
model=DecisionTreeClassifier()
model.fit(X_train, y_train)

# Evaluation

In [ ]:
y_pred = model.predict(X_test)


In [ ]:
accuracy_score(y_test, y_pred)

In [ ]:
precision_score(y_test, y_pred)

In [ ]:
recall_score(y_test, y_pred)

In [ ]:
f1_score(y_test, y_pred)

# K-Folds Cross Validation

In [ ]:
model=DecisionTreeClassifier()
scores = cross_validate(model, X, y, cv=3, scoring=('accuracy', 'average_precision', 'recall','f1'))

In [ ]:
scores

# Random Forest

In [ ]:
rfc = RandomForestClassifier()
rfc_scores = cross_validate(rfc, X, y.values.ravel(), cv=3, scoring=('accuracy', 'average_precision', 'recall','f1'))

In [ ]:
rfc_scores

In [ ]:
report = pd.DataFrame(index=list(rfc_scores.keys())[2:], columns=['Decision Tree', 'Random Forest'])
for key in report.index:
  report.loc[key] = [np.mean(scores[key]), np.mean(rfc_scores[key])]

report['Improvment'] = report['Random Forest'] - report['Decision Tree']
report *= 100
report = report.astype(float).round(1)

In [ ]:
report

# Feature Importance

In [ ]:
rfc = RandomForestClassifier()
rfc.fit(X_train, y_train.values.ravel())
importance = rfpimp.importances(rfc, X_test, y_test)
rfpimp.plot_importances(importance)

# Feature Engineering